## Module 10 Assignment: Web Scraping with Selenium
* Author: Venkat sai surya varaprasad Rao Athuluru
* version 1.0

## References
* https://www.programiz.com/python-programming/working-csv-files
* https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
* https://realpython.com/python-boto3-aws-s3/
* https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 

## Step 1: Loading Modules

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install awscli

Note: you may need to restart the kernel to use updated packages.


## Step 2: Scraping the website

In [4]:
###Load modules
#!pip install webdriver-manager
#!pip install awscli
import awscli
import boto3
import selenium
import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

####SCRAPE THE WEBSITE######
###call the webdriver
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]') #identifies the location of the EIN element
inputElement.send_keys('0') #sends the "0" as the search value for EIN 
inputElement1 = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]').click() #instatiates the click of the search
sleep(4) #allow for the page to load by adding a sleep element
#identify the table to scrape
table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
sleep(1)
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table

#loop through dataframe to export table
for row in table.find_elements(By.CSS_SELECTOR,'tr'):
      cols = df.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])
      
#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
df #let's have a look at the data before creating the CSV file and loading it into s3

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,None,None,None,None,None,None
1,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
2,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
3,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
4,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
5,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
6,"""Y"" Dive, Inc.",48-45-01,854252095,NFP,SAINT ALBANS,NY
7,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
8,#FeedHamburg,48-37-35,854150318,NFP,HAMBURG,NY
9,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY


## Step 3: Data cleaning

Data cleaning is performed because after scraping the website in the first csv, data starts with zero indexing, so to remove that and make it into one we are performing this step.

In [5]:
dff = df[1:]
dff

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
1,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
2,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
3,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
4,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
5,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
6,"""Y"" Dive, Inc.",48-45-01,854252095,NFP,SAINT ALBANS,NY
7,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
8,#FeedHamburg,48-37-35,854150318,NFP,HAMBURG,NY
9,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
10,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA


## Step 4: Creating S3 bucket

In [6]:
import logging
import boto3
from botocore.exceptions import ClientError

def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region. If a region is not specified,
    the bucket is created in the S3 default region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-east-2'
    :return: True if bucket created, else False
    """
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

# Attempt to create the bucket
bucket_name = 'venkatathuluru-m10'  # Ensure this is globally unique and all lowercase
region = 'us-east-2'  # Specify your region

if create_bucket(bucket_name, region):
    print(f'Bucket "{bucket_name}" created successfully.')
else:
    print(f'Failed to create bucket "{bucket_name}". Please check the logs for more details.')


Bucket "venkatathuluru-m10" created successfully.


## Step 5: Uploading the scraped data into s3 bucket

In [7]:
import awscli
import pandas as pd
from io import StringIO  # For handling CSV content as a string buffer
from datetime import datetime 



# Assuming 'df' is your final DataFrame
csv_buffer = StringIO()
df.to_csv(csv_buffer)

# Generate timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Initialize an S3 client using boto3
s3_resource = boto3.resource('s3')
bucket_name = 'venkatathuluru-m10'  # Replace with your bucket name
file_name = f'charities_bureau_scrape_{timestamp}.csv'  # Append timestamp to file name

# Upload the csv locally
df.to_csv(file_name)

# Upload the CSV file
s3_resource.Object(bucket_name, file_name).put(Body=csv_buffer.getvalue())

print(f" Succesfully File uploaded to s3 bucket: {bucket_name}")

 Succesfully File uploaded to s3 bucket: venkatathuluru-m10


## Step 6: Uploading the Updated data into S3 bucket

In [8]:
import awscli
import boto3
import pandas as pd
from io import StringIO  # For handling CSV content as a string buffer

# Assuming 'df' is your final DataFrame
csv_buffer = StringIO()
dff.to_csv(csv_buffer)

# Initialize an S3 client using boto3
s3_resource = boto3.resource('s3')
bucket_name = 'venkatathuluru-m10'  # Replace with your bucket name
file_name = 'updated_venkat_charities_bureau_scrape.csv'  # Replace with your desired file name
# Upload the csv locally

dff.to_csv(file_name)

# Upload the CSV file
s3_resource.Object(bucket_name, file_name).put(Body=csv_buffer.getvalue())

print(f"File {file_name} uploaded to {bucket_name}")


File updated_venkat_charities_bureau_scrape.csv uploaded to venkatathuluru-m10
